In [1]:
import sys
import os

# Add the src folder to the system path
sys.path.append(os.path.abspath("../src"))

In [2]:
import os
from tqdm.notebook import tqdm
from openai import OpenAI
from dotenv import load_dotenv

from chatgpt_tools.prompts import translate_word, generate_sentence, dictionarize_word
from chatgpt_tools.tts import get_audio
from anki_tools.anki_deck_creator import AnkiDeckManager

load_dotenv()
client = OpenAI()

BASE_DIR = "/mnt/d/OneDrive/Projects/data/danish"
AUDIO_DIR = os.path.join(BASE_DIR, "anki_audio")
DB_PATH = os.path.join(BASE_DIR, "cards.db")

os.makedirs(AUDIO_DIR, exist_ok=True)

def process_word_batch(word, language="Danish"):
    """
    Process a single word the same way as the Gradio interface would.
    """
    translation = translate_word(client, word, source_lang=language)
    sentence = generate_sentence(client, word, language)
    dictionary_form = dictionarize_word(client, word, language)
    audio_path = get_audio(client, word, language, output_dir=AUDIO_DIR)

    return word, translation, sentence, audio_path, dictionary_form

def process_word_list(word_list, language="Danish"):
    """
    Process a list of words, add them to the database, and export the deck.
    """
    deck_manager = AnkiDeckManager(
        deck_name="Danish Learning Deck",
        model_id=1607392319,
        model_name="Danish with Audio",
        db_path=DB_PATH
    )

    for word in tqdm(word_list, desc="Processing words"):
        danish_word, translation, sentence, audio_path, dictionary_form = process_word_batch(word, language)
        
        deck_manager.add_card(
            front=f"<div>{danish_word}</div><div>{sentence}</div>",
            back=f"<div>{translation}</div><div>{dictionary_form}</div>",
            front_audio_path=audio_path,
            back_audio_path="",
        )
        deck_manager.add_card(
            front=f"<div>{translation}</div>",
            back=f"<div>{danish_word}</div><div>{sentence}</div><div>{dictionary_form}</div>",
            front_audio_path="",
            back_audio_path=audio_path,
        )

    # Export the deck
    output_path = os.path.join(BASE_DIR, "danish_deck.apkg")
    deck_manager.export_to_apkg(output_path)
    print(f"Deck exported successfully to {output_path}")

In [3]:
danish_words = [
    "en enke",
    "et billede",
    "et ægteskab",
    "Det tror jeg.",
    "Det håber jeg.",
    "et menneske",
    "at tage afsted",
    "jeg tager aldrig brusebad",
    "jeg er tit sulten...",
    "at gøre rent",
    "at være er var været",
    "at have har havde haft",
    "Han bor sammen med sin kone.",
    "Han bor sammen med hans kone.",
    "Hendes mand kommer i Budapest.",
    "et barn børn",
    "søskende",
    "bedstefar, bedstemor, bedsteforældre",
    "Lejlighed",
    "så tit",
    "at være gift - skilt",
    "at læse",
    "at lære",
    "at studere",
    "at savne",
    "at tænke på",
    "en nationalitet",
    "et modersmal",
    "at være er",
    "at bo bor",
    "en sal",
    "til højre",
    "til venstre",
    "et telefonnumer",
    "at tro - jeg tror",
    "at synes - jeg synes",
    "at slappe af"
]

process_word_list(danish_words)

Database file /mnt/d/OneDrive/Projects/data/danish/cards.db does not exist. Creating...


Processing words:   0%|          | 0/37 [00:00<?, ?it/s]